In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()

from metapack_build.core import alt_col_name
import re



In [2]:

pkg = mp.jupyter.open_source_package()
pkg

`hudexchange.info-hic-facility-1` Last Update: 2019-07-09T23:34:01

__




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [hic](python:metapack_build.rowgenerator#copy_reference_group)**. 

## References

* **[hic_2018](https://www.hudexchange.info/resources/documents/2018-Housing-Inventory-Count-Raw-File.xlsx)**. 2018 HIC (Raw File)
* **[hic_2017](https://www.hudexchange.info/resources/documents/2017-Housing-Inventory-Count-Raw-File.xlsx)**. 2017 HIC (Raw File)
* **[hic_2016](https://www.hudexchange.info/resources/documents/2016-Housing-Inventory-Count-Raw-File.xlsx)**. 2016 HIC (Raw File)
* **[hic_2015](https://www.hudexchange.info/resources/documents/2015-Housing-Inventory-Count-Raw-File.xlsx)**. 2015 HIC (Raw File)
* **[hic_2014](https://www.hudexchange.info/resources/documents/2014-Housing-Inventory-Count-Raw-File.xlsx)**. 2014 HIC (Raw File)
* **[hic_2013](https://www.hudexchange.info/resources/documents/2013-Housing-Inventory-Count-Raw-File.xlsx)**. 2013 HIC (Raw File)
* **[hic_2012](https://www.hudexchange.info/resources/documents/2012HIC_RawData.xlsx#1)**. 2012 HIC (Raw File)

In [3]:
frames = {}
for r in pkg.references():
    f = r.dataframe()
    if not 'year' in f.columns:
        f.insert(0,'year', value=int(r.Year))
    frames[int(r.Year)]  = f


In [4]:
# Sort the frames by number of 
sorted_frames = {}
for s,k,v in reversed(sorted( (len(v.columns), k, v) for k,v in frames.items())):
    print(s,k)
    sorted_frames[k] = v
    

96 2018
82 2017
69 2016
47 2015
40 2014
39 2013
34 2012


In [11]:
def munge_cols(df):
    return [re.sub(r'_\d\d\d\d', '', alt_col_name(c,i)) for i, c in enumerate(df.columns) ]

from metapack_build.core import alt_col_name

# Create a list of all columns, apptempting to preserve order
all_cols  = list(map(alt_col_name, list(sorted_frames.values())[0].columns))
for f in list(sorted_frames.values())[1:]:
    all_cols +=  [ c for c in map(alt_col_name,f.columns) if c not in all_cols]
    

In [26]:
cm_series = { 'index': all_cols} # column map series
for k,f in frames.items():
    s = [None] * len(all_cols)
    for c in f.columns:
        alt = alt_col_name(c)
        s[all_cols.index(alt)] = c
    cm_series[k] = s

In [28]:
df = pd.DataFrame(cm_series)
df.to_csv('colmap.csv')

In [ ]:

cols = []
extra_cols = set()

# Check that the latest file has all of the columns that are in all of the earlier files
for f in frames.values():
    
    if not cols:
        cols = munge_cols(f)
    else:
        c1 = set(cols)
        c2 = set(munge_cols(f))
        extra_cols |= (c2-c1)
    
#for f in reversed(frames):
#    f.columns = munge_cols(f)
   

In [ ]:
df = pd.concat(frames, sort=True)

In [ ]:
str_cols = ['coc_number', 'coc_name', 'coc_category']
num_cols = [c for c in df.columns if c not in str_cols+['year']]
new_cols = ['year'] + str_cols + num_cols

In [ ]:
# Get rid of the blanks and '.' that prevent most of the columns from being numbers 
strings = set()
for c in num_cols:
    s = df[c]
    strings |=set(s[s.apply(lambda x: isinstance(x, str))])
    
    df[c] = pd.to_numeric(df[c],errors='coerce')
 
df = df[new_cols].sort_values(['year','coc_number'])

In [ ]:
df.head().T